### Memo

- Haluaako bebu että listauksessa on mukana NaN eli tyhjät sanat (eli lause loppuu).

In [49]:
kayta_OneDrive_tiedostoja = False
listattavien_sanojen_maara = 10

ångestfylld_directory = "ångestfylld.csv"
zip_file_path = 'OneDrive_2024-01-22.zip'
csv_directory_within_zip = 'korpus ångest'

In [50]:
import zipfile
import io
import pandas as pd
import numpy as np

# Lue tiedostot

In [51]:
def read_zip_directory():
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        file_list = [f for f in zip_file.namelist() if f.startswith(csv_directory_within_zip)]
        combined_df = pd.DataFrame()

        for file_name in file_list:
            with zip_file.open(file_name) as csv_file:
                df = pd.read_csv(io.TextIOWrapper(csv_file, 'utf-8'))
                combined_df = pd.concat([combined_df, df], ignore_index=True)
    combined_df = combined_df[['left context', 'right_context', 'text_date']]
    return combined_df

In [52]:
def read_ångestfylld_csv():
    df = pd.read_csv(ångestfylld_directory)
    df = df[['left context', 'right_context', 'text_date']]
    return df

In [53]:
if kayta_OneDrive_tiedostoja:
    df = read_zip_directory()
else:
    df = read_ångestfylld_csv()
df

,left context,right_context,text_date
0,Jag kommer aldrig glömma åren av orolig och,väntan på våra barn och avundas inte er som st...,2012-03-07 15:23:34
1,"Tänk att jag sitter här , efter 6 år av längta...",dagar och nätter i bagaget och plötsligt är ly...,2009-08-19 22:34:20
2,Inget att oroa sig för alltså Att välja land -,", "" har vi valt rätt ? """,2006-03-09 19:57:56
3,Att leva i ovisshet är naturligtvis,"och oroligt , man är så nära lyckan men risker...",2010-04-06 20:37:23
4,Det kan ju mycket väl vara genant och eller,att erkänna att man frivilligt var tvungen att...,2006-03-30 11:53:04
...,...,...,...
4632,NaN,NaN,NaN
4633,NaN,NaN,NaN
4634,NaN,NaN,NaN
4635,NaN,NaN,NaN


# Poista rivit joissa tyhjiä soluja

In [54]:
print(df.isna().sum())
print('length:', len(df))
df = df.dropna(axis=0) #drop rows with missing values + the last info rows from every csv file
print(df.isna().sum())
print('length:', len(df))

left context     74
right_context    45
text_date         8
dtype: int64
length: 4637
left context     0
right_context    0
text_date        0
dtype: int64
length: 4538


# Luo sarake vuosiluvuille

In [55]:
df['text_date'] = pd.to_datetime(df['text_date'])
df['year'] = df['text_date'].dt.year
df = df.drop(columns=['text_date'])
print(df['year'])
print(df.columns)

0       2012
1       2009
2       2006
3       2010
4       2006
        ... 
4624    2008
4625    2012
4626    2017
4627    2022
4628    2021
Name: year, Length: 4538, dtype: int64
Index(['left context', 'right_context', 'year'], dtype='object')


/tmp/ipykernel_89412/1931667789.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_date'] = pd.to_datetime(df['text_date'])
/tmp/ipykernel_89412/1931667789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['text_date'].dt.year


# Luo sarake yksittäisille sanoille

In [56]:
df['left_context_last_word'] = df['left context'].str.split().str[-1]
df[['left context', 'left_context_last_word']]

,left context,left_context_last_word
0,Jag kommer aldrig glömma åren av orolig och,och
1,"Tänk att jag sitter här , efter 6 år av längta...",dessa
2,Inget att oroa sig för alltså Att välja land -,-
3,Att leva i ovisshet är naturligtvis,naturligtvis
4,Det kan ju mycket väl vara genant och eller,eller
...,...,...
4624,Fiji : Jag tänkte jättemycket på det när jag v...,mest
4625,Denna hemska,hemska
4626,"Har sprungit på toa konstant senaste dagarna ,...",och
4627,Tiden till vul i vecka 8 känns som en lång spänd,spänd


In [57]:
df['right_context_first_word'] = df['right_context'].str.split().str[0]
df[['right_context', 'right_context_first_word']]

,right_context,right_context_first_word
0,väntan på våra barn och avundas inte er som st...,väntan
1,dagar och nätter i bagaget och plötsligt är ly...,dagar
2,", "" har vi valt rätt ? """,","
3,"och oroligt , man är så nära lyckan men risker...",och
4,att erkänna att man frivilligt var tvungen att...,att
...,...,...
4624,syner jag kunde ha .,syner
4625,väntan ...,väntan
4626,inför morgondagen ....,inför
4627,väntan .,väntan


# Jos valittu sana on erikoismerkki, valitse kontekstista seuraava sana

In [58]:
def extract_last_word(text):
    words = text.split()
    for word in reversed(words):
        if any(char.isalpha() for char in word):
            return word
    return np.nan
        
def extract_first_word(text):
    words = text.split()
    for word in words:
        if any(char.isalpha() for char in word):
            return word
    return np.nan

# Apply the function to the 'left context' column
df['left_context_testfunction'] = df['left context'].apply(extract_last_word)
df['right_context_testfunction'] = df['right_context'].apply(extract_first_word)

df.head(50)

,left context,right_context,year,left_context_last_word,right_context_first_word,left_context_testfunction,right_context_testfunction
0,Jag kommer aldrig glömma åren av orolig och,väntan på våra barn och avundas inte er som st...,2012,och,väntan,och,väntan
1,"Tänk att jag sitter här , efter 6 år av längta...",dagar och nätter i bagaget och plötsligt är ly...,2009,dessa,dagar,dessa,dagar
2,Inget att oroa sig för alltså Att välja land -,", "" har vi valt rätt ? """,2006,-,",",land,har
3,Att leva i ovisshet är naturligtvis,"och oroligt , man är så nära lyckan men risker...",2010,naturligtvis,och,naturligtvis,och
4,Det kan ju mycket väl vara genant och eller,att erkänna att man frivilligt var tvungen att...,2006,eller,att,eller,att
5,Att tänka på barn är inte längre,och stressande - det känns fantastiskt att tän...,2007,längre,och,längre,och
6,Stressad,småbarnsmor,2016,Stressad,småbarnsmor,Stressad,småbarnsmor
7,"Eller kommer du känba dig orolig ,",", ensam och sviken ?",2017,",",",",orolig,ensam
8,Nu känner jag mig mer,"än vad jag gjort på flera år , får panikångest...",2016,mer,än,mer,än
10,Finns det konflikter och mobbning på arbetspla...,o sömnlös så spelar det ingen större roll vad ...,2016,dig,o,dig,o


# Tässä listattuna osa valituista sanoista, joissa erikoismerkkejä

In [59]:
column_name = 'left_context_testfunction'

# Use a regular expression to filter cells with non-letter characters
non_letter_cells = df[df[column_name].str.contains('[^a-zA-ZåäöÅÄÖ]', na=False)]

# Display the result
print("Cells with non-letter characters:")
non_letter_cells[column_name].head(60)

Cells with non-letter characters:


1608           !Vad
4007    blod.Livräd
Name: left_context_testfunction, dtype: object

# Tarkistus että kaikki toimii niinkuin pitää

In [60]:
# manual function validation

import string

all_letters = string.ascii_letters

mask = df['right_context_first_word'].apply(lambda x: any(char not in all_letters for char in x))

# Use the mask to filter rows with special characters
rows_with_special_characters = df[mask]

# Print the resulting DataFrame
rows_with_special_characters.head(50)

,left context,right_context,year,left_context_last_word,right_context_first_word,left_context_testfunction,right_context_testfunction
0,Jag kommer aldrig glömma åren av orolig och,väntan på våra barn och avundas inte er som st...,2012,och,väntan,och,väntan
2,Inget att oroa sig för alltså Att välja land -,", "" har vi valt rätt ? """,2006,-,",",land,har
6,Stressad,småbarnsmor,2016,Stressad,småbarnsmor,Stressad,småbarnsmor
7,"Eller kommer du känba dig orolig ,",", ensam och sviken ?",2017,",",",",orolig,ensam
8,Nu känner jag mig mer,"än vad jag gjort på flera år , får panikångest...",2016,mer,än,mer,än
12,Inte verkade hon särskild,", inte verkade det som hon hade borderline el .",2010,särskild,",",särskild,inte
13,Mycket motvilligt och,arbetslös .,2013,och,arbetslös,och,arbetslös
14,Det här blev riktigt,.,2009,riktigt,.,riktigt,NaN
19,Jag är 21 och inget körkort och mina syskon är...,så vet jag att jag inte klarar det även om jag...,2020,och,så,och,så
20,Jag är 21 och inget körkort och mina syskon är...,så vet jag att jag inte klarar det även om jag...,2020,och,så,och,så


In [61]:
df.columns

Index(['left context', 'right_context', 'year', 'left_context_last_word',
       'right_context_first_word', 'left_context_testfunction',
       'right_context_testfunction'],
      dtype='object')

# Sarakkeiden siistimistä

In [62]:
df = df.drop(columns=['left_context_last_word', 'right_context_first_word',])
df['left_context_last_word'] = df['left_context_testfunction']
df['right_context_first_word'] = df['right_context_testfunction']
df.head(50)

,left context,right_context,year,left_context_testfunction,right_context_testfunction,left_context_last_word,right_context_first_word
0,Jag kommer aldrig glömma åren av orolig och,väntan på våra barn och avundas inte er som st...,2012,och,väntan,och,väntan
1,"Tänk att jag sitter här , efter 6 år av längta...",dagar och nätter i bagaget och plötsligt är ly...,2009,dessa,dagar,dessa,dagar
2,Inget att oroa sig för alltså Att välja land -,", "" har vi valt rätt ? """,2006,land,har,land,har
3,Att leva i ovisshet är naturligtvis,"och oroligt , man är så nära lyckan men risker...",2010,naturligtvis,och,naturligtvis,och
4,Det kan ju mycket väl vara genant och eller,att erkänna att man frivilligt var tvungen att...,2006,eller,att,eller,att
5,Att tänka på barn är inte längre,och stressande - det känns fantastiskt att tän...,2007,längre,och,längre,och
6,Stressad,småbarnsmor,2016,Stressad,småbarnsmor,Stressad,småbarnsmor
7,"Eller kommer du känba dig orolig ,",", ensam och sviken ?",2017,orolig,ensam,orolig,ensam
8,Nu känner jag mig mer,"än vad jag gjort på flera år , får panikångest...",2016,mer,än,mer,än
10,Finns det konflikter och mobbning på arbetspla...,o sömnlös så spelar det ingen större roll vad ...,2016,dig,o,dig,o


In [63]:
df = df.drop(columns=['left_context_testfunction', 'right_context_testfunction'])
df.columns

Index(['left context', 'right_context', 'year', 'left_context_last_word',
       'right_context_first_word'],
      dtype='object')

# Laske sanat ja näytä tulokset

In [64]:
def count_words(dataframe, year):
    left_context = {}
    right_context = {}

    for index, row in dataframe.iterrows():
        if row['year'] == year:
            if row['left_context_last_word'] in left_context:
                left_context[row['left_context_last_word']] += 1
            else:
                left_context[row['left_context_last_word']] = 1
            if row['right_context_first_word'] in right_context:
                right_context[row['right_context_first_word']] += 1
            else:
                right_context[row['right_context_first_word']] = 1
    return left_context, right_context

years = df['year'].unique()
sorted_years = np.sort(years)[::-1]

for year in sorted_years:
    left_context, right_context = count_words(df, year)
    sorted_left_context = sorted(left_context.items(), key=lambda item: item[1], reverse=True)
    sorted_right_context = sorted(right_context.items(), key=lambda item: item[1], reverse=True)
    
    print(f'Vuosi {year}:')
    print('Vasen konteksti | Oikea konteksti')
    for i in range(listattavien_sanojen_maara):
        try:
            print(f'{sorted_left_context[i][0]}: {sorted_left_context[i][1]}        {sorted_right_context[i][0]}: {sorted_right_context[i][1]}')
        except IndexError:
            continue
    print()

Vuosi 2022:
Vasen konteksti | Oikea konteksti
och: 21        nan: 13
så: 9        för: 8
väldigt: 6        och: 7
något: 2        att: 5
där: 2        när: 3
bli: 2        så: 2
det: 2        innan: 2
rädd: 2        över: 2
eller: 2        om: 2
mig: 2        eftersom: 2

Vuosi 2021:
Vasen konteksti | Oikea konteksti
och: 14        nan: 16
en: 7        och: 8
väldigt: 6        för: 4
så: 5        person: 3
orolig: 3        med: 3
bara: 3        att: 2
mer: 3        av: 2
bli: 2        när: 2
hur: 2        det: 2
eller: 2        orolig: 1

Vuosi 2020:
Vasen konteksti | Oikea konteksti
och: 36        nan: 17
så: 6        och: 16
en: 5        att: 7
mer: 4        så: 4
vara: 3        av: 4
Kränkande: 3        över: 3
väldigt: 2        när: 3
otroligt: 2        för: 2
spinkiga: 2        tid: 2
blir: 1        har: 2

Vuosi 2019:
Vasen konteksti | Oikea konteksti
och: 35        nan: 23
så: 9        och: 17
är: 5        för: 8
stressad: 5        att: 5
blir: 4        inför: 4
väldigt: 4      